# Chapter 35: GitOps with ArgoCD

Traditional CI/CD pipelines often push changes to Kubernetes clusters using imperative commands like `kubectl apply` or Helm installations. This approach creates operational challenges: configuration drift, limited audit trails, and difficulty determining the actual state of the cluster versus the intended state. GitOps addresses these challenges by treating Git repositories as the single source of truth for declarative infrastructure and application definitions.

ArgoCD is a declarative, Kubernetes-native continuous delivery tool that implements the GitOps methodology. It automates the deployment of desired application states defined in Git repositories to specified target environments, providing a comprehensive solution for configuration management, drift detection, and automated remediation. This chapter explores how ArgoCD transforms Kubernetes deployments from imperative scripts to a Git-centric, automated workflow.

## 35.1 GitOps Principles

GitOps is an operational paradigm that applies DevOps best practices used for application development to infrastructure automation. The core philosophy centers on Git as the single source of truth for declarative infrastructure and applications.

### The Four Pillars of GitOps

**1. Declarative Configuration**
All system state is defined declaratively and stored in version control. Rather than scripting imperative steps to reach a desired state, you describe the end state (Kubernetes manifests, Helm charts, Kustomize overlays) and the system converges to that state.

```yaml
# Example: Declarative application definition in Git
apiVersion: apps/v1
kind: Deployment
metadata:
  name: payment-service
spec:
  replicas: 3
  selector:
    matchLabels:
      app: payment-service
  template:
    spec:
      containers:
      - name: payment
        image: company/payment-service:v2.1.0
        resources:
          requests:
            memory: "512Mi"
            cpu: "250m"
```

**2. Version Control as Single Source of Truth**
Every change to the system is recorded as a Git commit, providing an immutable audit trail. Rollbacks become `git revert` operations; rollouts are `git push` events. This eliminates "works on my machine" problems and configuration sprawl.

**3. Automated Synchronization**
Automated agents (ArgoCD controllers) continuously monitor the Git repository and the live cluster state. When divergence is detected, the agent automatically applies changes to bring the cluster into the desired state defined in Git.

**4. Continuous Reconciliation and Drift Detection**
The system continuously ensures the actual state matches the desired state. Manual changes made directly to the cluster (kubectl apply) are flagged as drift and optionally automatically corrected, preventing configuration rot.

### GitOps vs. Traditional CI/CD

| Aspect | Traditional CI/CD | GitOps |
|--------|------------------|---------|
| **State Definition** | Pipeline scripts and manual steps | Declarative configs in Git |
| **Deployment Trigger** | CI pipeline completion events | Git commit or automated sync |
| **Rollback Method** | Re-run pipeline with old version | Git revert and automatic sync |
| **Audit Trail** | CI/CD logs (often ephemeral) | Git history (immutable) |
| **Drift Handling** | Manual detection and correction | Automatic detection and self-healing |
| **Security Model** | CI/CD needs cluster credentials | Cluster pulls from Git (read-only) |

## 35.2 ArgoCD Architecture

ArgoCD operates as a Kubernetes controller following the standard operator pattern. It extends the Kubernetes API with Custom Resource Definitions (CRDs) that define applications and their synchronization behaviors.

### Core Components

**API Server (argocd-server)**
- Serves the ArgoCD API and web UI
- Handles authentication and authorization
- Processes Git webhook events
- Exposes gRPC and REST APIs for CLI and UI interactions

**Repository Server (argocd-repo-server)**
- Maintains local clones of Git repositories
- Generates Kubernetes manifests from Helm, Kustomize, or plain YAML
- Handles Git credential management
- Caches generated manifests to improve performance

**Application Controller (argocd-application-controller)**
- Monitors Application CRDs
- Compares live cluster state with target state from Git
- Executes synchronization operations
- Handles pruning of resources no longer defined in Git

**Dex (optional)**
- OpenID Connect (OIDC) provider for SSO integration
- Supports LDAP, SAML, GitHub, GitLab, and other identity providers

**Redis**
- Caching layer for session management and temporary data
- Stores rendered manifests and state information

### Custom Resource Definitions

ArgoCD introduces three primary CRDs:

**Application**
The fundamental unit of deployment management, linking a Git source to a destination cluster and namespace.

```yaml
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: production-payment-api
  namespace: argocd
spec:
  project: production
  source:
    repoURL: https://github.com/company/gitops-repo.git
    targetRevision: main
    path: apps/payment-api/overlays/production
  destination:
    server: https://kubernetes.default.svc
    namespace: payment
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
```

**AppProject**
Provides logical grouping, access control, and resource constraints for applications.

```yaml
apiVersion: argoproj.io/v1alpha1
kind: AppProject
metadata:
  name: production
  namespace: argocd
spec:
  description: Production workloads
  sourceRepos:
    - https://github.com/company/gitops-repo.git
  destinations:
    - namespace: payment
      server: https://kubernetes.default.svc
    - namespace: user-management
      server: https://kubernetes.default.svc
  clusterResourceWhitelist:
    - group: ""
      kind: Namespace
  namespaceResourceBlacklist:
    - group: ""
      kind: ResourceQuota
  orphanedResources:
    warn: true
```

**ApplicationSet**
Enables templating and multi-cluster/multi-tenant deployments by generating multiple Application resources from a single manifest.

### Synchronization Flow

1. **Detection**: Controller detects a new Git commit or manual refresh request
2. **Manifest Generation**: Repo-server clones the repository and generates manifests ( Helm template, kustomize build, etc.)
3. **State Comparison**: Controller compares generated manifests with live cluster resources using `kubectl diff` logic
4. **Sync Execution**: If differences exist, controller applies changes using server-side apply
5. **Health Assessment**: Controller monitors resource health (Deployment rollout, Service endpoints, Custom health checks)
6. **Status Update**: Application CR status is updated with sync state, health, and operation information

## 35.3 Installation and Setup

ArgoCD can be installed using standard Kubernetes manifests, Helm, or operators. The namespace-scoped installation is recommended for most environments.

### Core Installation

Install the latest stable version:

```bash
# Create namespace
kubectl create namespace argocd

# Install ArgoCD
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

# Verify installation
kubectl wait --for=condition=available deployment -n argocd argocd-server --timeout=300s
```

This installs:
- ArgoCD server (UI/API)
- Application controller
- Repo server
- Dex (SSO)
- Redis

### High Availability Installation

For production environments, use the HA manifest which includes:
- Multiple replicas of API server and repo-server
- Redis Sentinel for high availability
- Horizontal Pod Autoscaler configurations

```bash
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/ha/install.yaml
```

### Accessing the UI

By default, ArgoCD exposes the UI via ClusterIP. Access methods include:

**Port Forwarding (Development):**
```bash
kubectl port-forward svc/argocd-server -n argocd 8080:443
# Access https://localhost:8080
```

**LoadBalancer (Cloud):**
```bash
kubectl patch svc argocd-server -n argocd -p '{"spec": {"type": "LoadBalancer"}}'
```

**Ingress (Production):**
```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: argocd-server-ingress
  namespace: argocd
  annotations:
    nginx.ingress.kubernetes.io/backend-protocol: "HTTPS"
    nginx.ingress.kubernetes.io/ssl-passthrough: "true"
    cert-manager.io/cluster-issuer: "letsencrypt-prod"
spec:
  ingressClassName: nginx
  rules:
  - host: argocd.company.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: argocd-server
            port:
              number: 443
  tls:
  - hosts:
    - argocd.company.com
    secretName: argocd-tls
```

### Initial Authentication

Retrieve the initial admin password:

```bash
# Get initial password
kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d

# Login via CLI
argocd login localhost:8080 --username admin --password <initial-password>

# Change password
argocd account update-password --current-password <initial> --new-password <secure-password>
```

### CLI Installation

Install the ArgoCD CLI for automation and scripting:

```bash
# macOS
brew install argocd

# Linux
curl -sSL -o argocd-linux-amd64 https://github.com/argoproj/argo-cd/releases/latest/download/argocd-linux-amd64
sudo install -m 555 argocd-linux-amd64 /usr/local/bin/argocd

# Verify
argocd version --client
```

## 35.4 Application Management

Applications are the primary abstraction in ArgoCD, representing a deployable unit consisting of source configuration, destination cluster, and synchronization policy.

### Application Specification

A complete Application manifest includes:

```yaml
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: user-service-prod
  namespace: argocd
  finalizers:
    - resources-finalizer.argocd.argoproj.io  # Ensures resources are deleted when app is deleted
  labels:
    environment: production
    team: platform
spec:
  # Project for access control
  project: production
  
  # Source configuration
  source:
    repoURL: https://github.com/company/gitops-repo.git
    targetRevision: v2.1.0  # Can be branch, tag, or commit SHA
    path: applications/user-service
    helm:
      valueFiles:
        - values-production.yaml
      parameters:
        - name: replicaCount
          value: "5"
        - name: image.tag
          value: "v2.1.0"
    kustomize:
      namePrefix: prod-
      commonLabels:
        env: production
  
  # Destination cluster and namespace
  destination:
    server: https://kubernetes.default.svc
    namespace: user-service
  
  # Sync policy configuration
  syncPolicy:
    automated:
      prune: true        # Delete resources not in Git
      selfHeal: true     # Correct drift automatically
      allowEmpty: false  # Prevent deleting all resources
    syncOptions:
      - CreateNamespace=true
      - PrunePropagationPolicy=foreground
      - PruneLast=true
      - RespectIgnoreDifferences=true
    retry:
      limit: 5
      backoff:
        duration: 5s
        factor: 2
        maxDuration: 3m
  
  # Ignore specific differences (e.g., dynamically injected fields)
  ignoreDifferences:
    - group: apps
      kind: Deployment
      jsonPointers:
        - /spec/replicas  # Ignore HPA-driven replica changes
  
  # Revision history limit
  revisionHistoryLimit: 10
```

### Supported Source Types

ArgoCD supports multiple configuration management tools:

**Plain YAML/JSON**
Direct Kubernetes manifests stored in Git.

**Helm**
```yaml
source:
  helm:
    chart: postgresql
    repoURL: https://charts.bitnami.com/bitnami
    targetRevision: 12.1.0
    values: |
      auth:
        postgresPassword: secret
      primary:
        persistence:
          size: 100Gi
```

**Kustomize**
```yaml
source:
  kustomize:
    namePrefix: prod-
    nameSuffix: -v1
    commonLabels:
      environment: production
    commonAnnotations:
      deployed-by: argocd
```

**Ksonnet, Jsonnet, and Custom Plugins**
ArgoCD supports extensible config management plugins for custom templating tools.

### App of Apps Pattern

Manage multiple applications using a parent Application that references other Application manifests:

```yaml
# bootstrap-app.yaml
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: app-of-apps
  namespace: argocd
spec:
  destination:
    namespace: argocd
    server: https://kubernetes.default.svc
  project: default
  source:
    path: bootstrap
    repoURL: https://github.com/company/gitops-repo.git
    targetRevision: HEAD
```

Directory structure:
```
bootstrap/
├── payment-service.yaml
├── user-service.yaml
├── notification-service.yaml
└── infrastructure/
    ├── ingress-nginx.yaml
    ├── cert-manager.yaml
    └── monitoring.yaml
```

This enables bootstrapping entire environments by applying a single Application.

### Application Projects

Projects provide boundaries for applications, controlling:
- Allowed source repositories
- Permitted destination clusters/namespaces
- Allowed resource kinds (prevent CRDs in certain projects)
- Role-based access control (RBAC)
- Resource quotas

```yaml
apiVersion: argoproj.io/v1alpha1
kind: AppProject
metadata:
  name: production-critical
  namespace: argocd
spec:
  description: Critical production services
  
  # Allowed Git sources
  sourceRepos:
    - https://github.com/company/production-apps.git
    - https://charts.helm.sh/stable
  
  # Allowed deployment targets
  destinations:
    - namespace: prod-critical
      server: https://production-cluster.company.com
  
  # Allow specific cluster-scoped resources
  clusterResourceWhitelist:
    - group: ""
      kind: Namespace
    - group: rbac.authorization.k8s.io
      kind: ClusterRole
  
  # Deny dangerous resources
  namespaceResourceBlacklist:
    - group: ""
      kind: ResourceQuota
    - group: ""
      kind: LimitRange
  
  # Enable orphaned resource monitoring
  orphanedResources:
    warn: true
    ignore:
      - kind: ConfigMap
        name: kube-root-ca.crt
```

## 35.5 Synchronization

Synchronization is the process of applying the desired state from Git to the live cluster. ArgoCD provides granular control over how and when synchronization occurs.

### Synchronization Strategies

**Automatic Sync**
Triggered by Git commits or scheduled intervals:
```yaml
syncPolicy:
  automated:
    prune: true      # Remove resources not in Git
    selfHeal: true   # Correct manual changes
```

**Manual Sync**
Requires explicit user or API call to trigger:
```yaml
# Default behavior - no automated section
syncPolicy: {}
```

Trigger manually:
```bash
argocd app sync user-service-prod
```

### Sync Waves and Hooks

ArgoCD respects Kubernetes resource hooks and custom sync waves to control deployment order.

**Sync Waves**
Assign numeric waves (0-100) to resources to control creation order:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: database
  annotations:
    argocd.argoproj.io/sync-wave: "1"
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-service
  annotations:
    argocd.argoproj.io/sync-wave: "2"
  # Depends on database
```

Resources sync in wave order: -10, -9 ... 0 ... 9, 10. Resources in the same wave sync in parallel.

**Resource Hooks**
Execute jobs at specific lifecycle phases:

```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: database-migration
  annotations:
    argocd.argoproj.io/hook: PreSync
    argocd.argoproj.io/hook-delete-policy: HookSucceeded
spec:
  template:
    spec:
      containers:
      - name: migrate
        image: company/api-service:v2.1.0
        command: ["npm", "run", "migrate"]
      restartPolicy: OnFailure
```

Hook types:
- `PreSync`: Before main resources
- `Sync`: With main resources
- `PostSync`: After main resources succeed
- `SyncFail`: If synchronization fails

Delete policies:
- `HookSucceeded`: Delete after success
- `HookFailed`: Delete after failure
- `BeforeHookCreation`: Delete before creating new hook

### Sync Options

Fine-tune synchronization behavior:

```yaml
syncPolicy:
  syncOptions:
    # Create namespace if it doesn't exist
    - CreateNamespace=true
    
    # Server-side apply for large CRDs
    - ServerSideApply=true
    
    # Validate resources before applying (kubectl --validate=true)
    - Validate=true
    
    # Prune propagation policy
    - PrunePropagationPolicy=foreground
    
    # Prune resources last (after creation)
    - PruneLast=true
    
    # Respect diffing customization
    - RespectIgnoreDifferences=true
    
    # Apply out of sync only (faster for large apps)
    - ApplyOutOfSyncOnly=true
```

## 35.6 Self-Healing

Self-healing is a core GitOps principle ensuring the cluster maintains the desired state defined in Git, automatically correcting manual changes or failures.

### Drift Detection

ArgoCD continuously compares the live cluster state with the desired Git state. Drift occurs when:
- Manual `kubectl apply` changes resources
- Controllers modify resources (HPA scaling, cert-manager injecting certificates)
- Node failures cause Pod rescheduling
- Network issues prevent resource reconciliation

**Viewing Drift:**
In the UI, drifted resources appear as "OutOfSync" with detailed diff views:

```bash
# CLI diff
argocd app diff user-service-prod

# Detailed resource status
kubectl describe application user-service-prod -n argocd
```

### Automatic Self-Healing

When enabled, ArgoCD automatically reverts drift:

```yaml
spec:
  syncPolicy:
    automated:
      selfHeal: true
```

Self-healing behavior:
1. Controller detects drift (every 3 minutes or on webhook trigger)
2. Compares live manifest with Git manifest
3. Applies Git version to overwrite live changes
4. Logs the remediation event
5. Optionally notifies (via webhooks or notifications)

**Caution:** Self-healing can conflict with controllers that manage their own resources. Use `ignoreDifferences` to exempt fields managed by other controllers:

```yaml
ignoreDifferences:
  - group: apps
    kind: Deployment
    jsonPointers:
      - /spec/replicas  # Ignore HPA changes
  - group: ""
    kind: Service
    jsonPointers:
      - /spec/selector  # Ignore external traffic manager changes
```

### Pruning

When resources are removed from Git, ArgoCD can automatically delete them from the cluster:

```yaml
syncPolicy:
  automated:
    prune: true
```

Pruning safeguards:
- **Resource Deletion Finalizers**: Ensures ordered deletion (delete database after application)
- **Confirmation**: UI requires typing resource name for manual deletion
- **Prune Last**: Delete resources after all creations succeed

```yaml
syncPolicy:
  syncOptions:
    - PruneLast=true
    - PrunePropagationPolicy=foreground  # Wait for graceful termination
```

## 35.7 Application of Changes

ArgoCD supports progressive delivery patterns including blue-green deployments, canary releases, and feature flags through integrations and custom resource hooks.

### Progressive Delivery with Argo Rollouts

Argo Rollouts is a Kubernetes controller and CRD that provides advanced deployment capabilities:

```yaml
apiVersion: argoproj.io/v1alpha1
kind: Rollout
metadata:
  name: payment-service
spec:
  replicas: 5
  strategy:
    canary:
      canaryService: payment-service-canary
      stableService: payment-service-stable
      trafficRouting:
        nginx:
          stableIngress: payment-service-ingress
          annotationPrefix: nginx.ingress.kubernetes.io
      steps:
        - setWeight: 10
        - pause: {duration: 2h}
        - setWeight: 25
        - pause: {duration: 1h}
        - setWeight: 50
        - pause: {duration: 1h}
        - setWeight: 100
  selector:
    matchLabels:
      app: payment-service
  template:
    spec:
      containers:
      - name: payment
        image: company/payment-service:v2.1.0
```

**Analysis and Promotion:**
Automated metric analysis determines promotion:

```yaml
steps:
  - setWeight: 25
  - analysis:
      templates:
        - templateName: success-rate
      args:
        - name: service-name
          value: payment-service-canary
```

### Blue-Green Deployments

```yaml
strategy:
  blueGreen:
    activeService: payment-service-active
    previewService: payment-service-preview
    autoPromotionEnabled: false  # Manual promotion gate
    scaleDownDelaySeconds: 600   # Keep old version for 10 minutes
```

### Sync Windows and Maintenance

Control when changes can be applied:

```yaml
apiVersion: argoproj.io/v1alpha1
kind: AppProject
metadata:
  name: production
spec:
  syncWindows:
    - kind: allow
      schedule: "0 22 * * *"  # Allow syncs at 10 PM
      duration: 2h
      applications:
        - "*-prod"
      namespaces:
        - "production"
    - kind: deny
      schedule: "0 0 * * *"   # Block syncs during backup window
      duration: 1h
```

### Multiple Sources (GitOps with Config Repositories)

ArgoCD 2.6+ supports multiple sources for a single application:

```yaml
spec:
  sources:
    - repoURL: https://github.com/company/app-repo.git
      targetRevision: v2.1.0
      path: manifests
    - repoURL: https://github.com/company/config-repo.git
      targetRevision: main
      path: overlays/production
      ref: values  # Reference name for Helm values
  destination:
    server: https://kubernetes.default.svc
    namespace: production
```

## 35.8 Multi-Cluster Management

ArgoCD excels at managing applications across multiple Kubernetes clusters from a single control plane.

### Cluster Registration

Add external clusters:

```bash
# Add cluster using kubeconfig context
argocd cluster add production-cluster-context

# Or manually via secret
kubectl apply -f - <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: production-cluster
  namespace: argocd
  labels:
    argocd.argoproj.io/secret-type: cluster
type: Opaque
stringData:
  name: production
  server: https://production-api.company.com
  config: |
    {
      "bearerToken": "<token>",
      "tlsClientConfig": {
        "insecure": false,
        "caData": "<base64-ca-cert>"
      }
    }
EOF
```

### ApplicationSet for Multi-Cluster

ApplicationSet generates applications for multiple clusters automatically:

```yaml
apiVersion: argoproj.io/v1alpha1
kind: ApplicationSet
metadata:
  name: payment-service
  namespace: argocd
spec:
  generators:
    - list:
        elements:
          - cluster: production-us
            url: https://prod-us.company.com
            namespace: payment
            replicaCount: 5
          - cluster: production-eu
            url: https://prod-eu.company.com
            namespace: payment
            replicaCount: 3
          - cluster: staging
            url: https://staging.company.com
            namespace: payment-staging
            replicaCount: 1
  template:
    metadata:
      name: '{{cluster}}-payment-service'
    spec:
      project: default
      source:
        repoURL: https://github.com/company/gitops-repo.git
        targetRevision: HEAD
        path: apps/payment-service
        helm:
          parameters:
            - name: replicaCount
              value: '{{replicaCount}}'
      destination:
        server: '{{url}}'
        namespace: '{{namespace}}'
      syncPolicy:
        automated:
          prune: true
          selfHeal: true
```

### Cluster Generators

Automatically discover clusters by labels:

```yaml
generators:
  - clusters:
      selector:
        matchLabels:
          environment: production
          region: us-east
      values:
        replicaCount: 5
```

### Git File Generators

Drive deployments from JSON/YAML files in Git:

```yaml
generators:
  - git:
      repoURL: https://github.com/company/config.git
      revision: HEAD
      files:
        - path: "cluster-configs/*.json"
template:
  spec:
    destination:
      server: '{{cluster.address}}'
```

## 35.9 Best Practices

### Security Best Practices

**1. Principle of Least Privilege**
Grant ArgoCD minimal permissions using namespaced roles:

```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: argocd-application-controller
  namespace: production
rules:
  - apiGroups: ["*"]
    resources: ["*"]
    verbs: ["get", "list", "watch", "create", "update", "patch", "delete"]
```

**2. Secret Management**
Never store secrets in Git. Use:
- Sealed Secrets
- External Secrets Operator
- HashiCorp Vault integration
- AWS Secrets Manager CSI driver

```yaml
# Reference external secrets in Helm values
source:
  helm:
    values: |
      existingSecret: production-db-credentials
```

**3. Repository Credentials**
Use SSH keys or deploy tokens with read-only access:

```bash
argocd repo add git@github.com:company/gitops-repo.git \
  --ssh-private-key-path ~/.ssh/id_rsa \
  --insecure-ignore-host-key
```

**4. RBAC Configuration**
Configure fine-grained access in `argocd-rbac-cm`:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: argocd-rbac-cm
  namespace: argocd
data:
  policy.default: role:readonly
  policy.csv: |
    p, role:developer, applications, *, */*, allow
    p, role:developer, exec, create, */*, allow
    g, platform-team, role:developer
```

### Operational Excellence

**1. Resource Monitoring**
Monitor ArgoCD components:
```bash
# Controller metrics
kubectl port-forward svc/argocd-metrics -n argocd 8082:8082
# Check reconciliation lag, sync failures, resource counts
```

**2. Backup and Disaster Recovery**
Backup the `argocd` namespace regularly, especially:
- Application CRs
- AppProject CRs
- Repository credentials (secrets)

**3. Monorepo vs. Polyrepo**
- **Monorepo**: Easier dependency management, atomic changes across apps
- **Polyrepo**: Better access control, independent versioning
- **Hybrid**: Infrastructure in monorepo, application configs in app repos

**4. Disaster Recovery**
Document recovery procedures:
1. Restore ArgoCD from backup
2. Reconnect to Git repositories
3. Trigger full synchronization to restore cluster state

### Debugging Common Issues

**Sync Failures:**
```bash
# Check application events
kubectl describe application <app-name> -n argocd

# View controller logs
kubectl logs -n argocd deployment/argocd-application-controller

# Check repo server logs for manifest generation issues
kubectl logs -n argocd deployment/argocd-repo-server
```

**Permission Errors:**
Ensure the ArgoCD service account has appropriate RBAC permissions in target namespaces.

**Webhook Failures:**
If Git webhooks aren't triggering syncs:
1. Verify webhook URL is accessible from Git provider
2. Check `argocd-server` logs for webhook receipt
3. Verify webhook secret matches configuration

---

## Chapter Summary and Preview

This chapter established the foundations of GitOps using ArgoCD, transitioning from traditional push-based CI/CD to a declarative, pull-based model where Git repositories serve as the single source of truth. We explored the four pillars of GitOps—declarative configuration, version control as truth, automated synchronization, and self-healing—and examined how ArgoCD's architecture (API server, repository server, and application controller) implements these principles through Kubernetes-native Custom Resources.

We detailed the installation and configuration of ArgoCD for both development and high-availability production environments, emphasizing secure access patterns and ingress configuration. The Application and AppProject CRDs provide the primary interfaces for defining deployment boundaries, access controls, and resource constraints, while the ApplicationSet controller enables scalable multi-cluster and multi-tenant deployments.

Synchronization strategies including sync waves, resource hooks, and automated pruning enable complex deployment orchestration with dependency management. The self-healing capabilities ensure configuration drift is detected and corrected automatically, maintaining the desired state defined in Git against manual changes or system failures. Progressive delivery patterns through Argo Rollouts integration support canary and blue-green deployments, while multi-cluster management capabilities allow centralized control of distributed systems.

**Key Takeaways:**
- GitOps separates the deployment mechanism from the CI pipeline, improving security by eliminating long-lived cluster credentials from CI/CD systems.
- Always enable automated self-healing and pruning in production to prevent configuration drift, but carefully configure `ignoreDifferences` for fields managed by other controllers like HPA.
- Use ApplicationSets for multi-cluster scenarios to avoid repetitive Application definitions and ensure consistency across environments.
- Implement strict Project boundaries to prevent privilege escalation and enforce organizational standards regarding allowed repositories and resource types.
- Never store plaintext secrets in Git; integrate with external secret management systems or use Sealed Secrets.

**Next Chapter Preview:**
Chapter 36: GitOps with Flux introduces the alternative GitOps implementation from the Cloud Native Computing Foundation. While ArgoCD employs a pull-based model with a centralized UI and API, Flux adopts a decentralized, controller-native approach that treats GitOps operations as standard Kubernetes reconciliations. We will explore Flux's component architecture (Source Controller, Kustomize Controller, Helm Controller, and Notification Controller), its declarative dependency management through health checks and readiness gates, and its native integration with Kustomize for configuration management. The chapter compares Flux's progressive delivery capabilities with Flagger, its multi-tenancy support through RBAC impersonation, and scenarios where Flux's lightweight, Git-native approach may be preferable to ArgoCD's comprehensive platform model.